In [5]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv("data/Churn_Modelling.csv")
df.sample(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
7377,7378,15592999,Reid,691,France,Female,40,0,115465.98,1,1,1,60622.61,0
9490,9491,15655171,Yermakova,624,France,Male,80,3,0.00,1,1,1,65801.44,0
6392,6393,15701352,Fanucci,611,Spain,Female,28,3,96381.68,2,1,0,181419.29,0
7777,7778,15638730,Macleod,711,France,Female,21,0,82844.33,2,0,1,1408.68,0
7902,7903,15613962,Kenechi,499,France,Female,38,9,0.00,2,0,1,183042.20,0


In [7]:
df = df.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [ ]:
#label encoding Gender
label_encoding_gender = LabelEncoder()
df["Gender"] = label_encoding_gender.fit_transform(df["Gender"])

In [17]:
# label encoding Geography
ohe_geo = OneHotEncoder()
ohe_geo_encoding = ohe_geo.fit(df[["Geography"]])
ohe_geo_encoding_transform = ohe_geo_encoding.transform(df[["Geography"]]).toarray()
ohe_geo_encoding_transform

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [25]:
geo_df =  pd.DataFrame(ohe_geo_encoding_transform,columns=ohe_geo_encoding.get_feature_names_out())

In [27]:
new_df = pd.concat([geo_df,df.drop("Geography",axis=1)],axis=1)

In [29]:
x = new_df.drop("EstimatedSalary",axis=1)
y = new_df["EstimatedSalary"]

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)


In [37]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)


In [42]:
# model
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential([
    Dense(64,activation="relu",input_shape=(x_train.shape[1],)),
    Dense(32,activation="relu"),
    Dense(1)
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 64)                832       
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [47]:
model.compile(optimizer="adam",loss="mean_absolute_error",metrics=["mae"])

early_Stopping_callback = EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)

In [46]:
import datetime
from tensorflow.keras.callbacks import TensorBoard

log_dir = "regressionlog/fit"+datetime.datetime.now().strftime("%Y%M%D-%H%M%S")
tf_callback =TensorBoard(log_dir=log_dir,histogram_freq=1)

In [48]:
history = model.fit(
    x_train_scaled,y_train,validation_data =(x_test_scaled,y_test),
    epochs=100, callbacks=[early_Stopping_callback,tf_callback]
)

Epoch 1/100


250/250 [==============================] - 12s 17ms/step - loss: 100370.3203 - mae: 100370.3203 - val_loss: 98489.0391 - val_mae: 98489.0391
Epoch 2/100
250/250 [==============================] - 2s 9ms/step - loss: 99507.3594 - mae: 99507.3594 - val_loss: 96725.7734 - val_mae: 96725.7734
Epoch 3/100
250/250 [==============================] - 3s 12ms/step - loss: 96411.0703 - mae: 96411.0703 - val_loss: 92178.3281 - val_mae: 92178.3281
Epoch 4/100
250/250 [==============================] - 3s 10ms/step - loss: 90347.0391 - mae: 90347.0391 - val_loss: 84722.3438 - val_mae: 84722.3438
Epoch 5/100
250/250 [==============================] - 2s 10ms/step - loss: 81714.1094 - mae: 81714.1094 - val_loss: 75498.9766 - val_mae: 75498.9766
Epoch 6/100
250/250 [==============================] - 2s 9ms/step - loss: 72060.4844 - mae: 72060.4844 - val_loss: 66344.0703 - val_mae: 66344.0703
Epoch 7/100
250/250 [==============================] - 2s 9ms/step - loss: 63228.9297 - mae: 6322

In [50]:
%load_ext tensorboard

In [51]:
%tensorboard --logdir regressionlog/fit20240212

In [52]:
test_loss,test_mae = model.evaluate(x_test_scaled,y_test)
test_mae


63/63 [==============================] - 0s 5ms/step - loss: 50211.8203 - mae: 50211.8203


50211.8203125

In [53]:
model.save("data/regression_model.h5")

e:\ANN\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
